In [1]:
# Carga de datos

import pandas as pd

test_data = pd.read_csv(
    "../files/input/test_data.csv.zip",
    index_col = False,
    compression = "zip",
)


train_data = pd.read_csv(
    "../files/input/train_data.csv.zip",
    index_col = False,
    compression = "zip",
)

In [2]:
# Paso 1.
# Preprocese los datos.
# - Cree la columna 'Age' a partir de la columna 'Year'.
#   Asuma que el año actual es 2021.
# - Elimine las columnas 'Year' y 'Car_Name'.

def clean_df(df):
    df['Age'] = 2021-df['Year']
    df = df.drop(columns=['Year','Car_Name'])

    return df

train_data = clean_df(train_data)

test_data = clean_df(test_data)

train_data.tail()

,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Age
206,2.55,4.43,57000,Petrol,Dealer,Manual,0,10
207,3.95,6.80,36000,Petrol,Dealer,Manual,0,6
208,0.10,0.75,92233,Petrol,Individual,Manual,0,15
209,6.00,7.60,77632,Diesel,Dealer,Manual,0,7
210,6.00,8.40,4000,Petrol,Dealer,Manual,0,5


In [3]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.

x_train = train_data.drop(columns="Present_Price")
y_train = train_data["Present_Price"]

x_test = test_data.drop(columns="Present_Price")
y_test = test_data["Present_Price"]

In [4]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Escala las variables numéricas al intervalo [0, 1].
# - Selecciona las K mejores entradas.
# - Ajusta un modelo de regresion lineal.
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression


categorical_features = ["Fuel_Type","Selling_type","Transmission"]
non_categoriacal_features = x_train.columns
non_categoriacal_features = non_categoriacal_features.drop(categorical_features).tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat',OneHotEncoder(),categorical_features),
        ('num',MinMaxScaler(),non_categoriacal_features)
    ],
    remainder="passthrough"
)

pipeline = Pipeline([
    ('preprocessor',preprocessor),
    ('feature_selection',SelectKBest(score_func=f_regression)),
    ('regression', LinearRegression())

])

In [5]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use el error medio absoluto
# para medir el desempeño modelo.
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, balanced_accuracy_score, precision_score, median_absolute_error

param_grid ={
    'feature_selection__k': range(1,15)
}


model = GridSearchCV(
    pipeline,
    param_grid,
    cv=10,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    refit=True
)


model.fit(x_train, y_train)


y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)


print("Mejores parámetros encontrados:", model.best_params_)
print("Mejor puntuación precision train:", model.best_score_)

Mejores parámetros encontrados: {'feature_selection__k': 11}
Mejor puntuación precision train: -8.300109997431655


In [6]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.
#

import os
import gzip
import pickle

models_dir = '../files/models'
os.makedirs(models_dir, exist_ok=True)

# Guardar el modelo comprimido en formato gzip
gzip_path = os.path.join(models_dir, "model.pkl.gz")
with gzip.open(gzip_path, "wb") as gz_file:
    pickle.dump(model, gz_file)

In [7]:
# Paso 6.
# Calcule las metricas r2, error cuadratico medio, y error absoluto medio
# para los conjuntos de entrenamiento y prueba. Guardelas en el archivo
# files/output/metrics.json. Cada fila del archivo es un diccionario con
# las metricas de un modelo. Este diccionario tiene un campo para indicar
# si es el conjunto de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'metrics', 'dataset': 'train', 'r2': 0.8, 'mse': 0.7, 'mad': 0.9}
# {'type': 'metrics', 'dataset': 'test', 'r2': 0.7, 'mse': 0.6, 'mad': 0.8}
#
import json
from sklearn.metrics import r2_score, median_absolute_error, mean_squared_error

def calculate_save_metrics(model, x_train, x_test, y_train, y_test):
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    metrics_train = {
        'type': 'metrics',
        'dataset': 'train',
        'r2': r2_score(y_train, y_train_pred),
        'mse': mean_squared_error(y_train, y_train_pred),
        'mad': median_absolute_error(y_train, y_train_pred),
    }


    metrics_test = {
        'type': 'metrics',
        'dataset': 'test',
        'r2': r2_score(y_test, y_test_pred),
        'mse': mean_squared_error(y_test, y_test_pred),
        'mad': median_absolute_error(y_test, y_test_pred),
    }

    output_dir = '../files/output'
    os.makedirs(output_dir, exist_ok=True)

    output_path = os.path.join(output_dir, 'metrics.json')
    with open(output_path, 'w') as f:
        f.write(json.dumps(metrics_train)+ '\n')
        f.write(json.dumps(metrics_test)+ '\n')

In [8]:
def main(model, x_train, x_test, y_train, y_test):

    import os
    #os.makedirs('../files/output', exist_ok=True)

    calculate_save_metrics(model, x_train, x_test, y_train, y_test)

main(model, x_train, x_test, y_train, y_test)